### Import libraries and set up Redshift connnection string

In [1]:
from sqlalchemy import create_engine
import psycopg2
import pandas as pd
import os

In [2]:
redshift_endpoint = os.getenv("REDSHIFT_ENDPOINT")
redshift_user = os.getenv("REDSHIFT_USER")
redshift_pass = os.getenv("REDSHIFT_PASS")
port = 5439
dbname = 'prod'

engine_str = "redshift+psycopg2://%s:%s@%s:%s/%s" % (redshift_user,redshift_pass,redshift_endpoint,str(port),dbname)

engine = create_engine(engine_str)

### Pull sample data from Redshift into a Pandas dataframe
show the first 5 rows of the data

In [4]:
mx_data = pd.read_sql_query('select * from edw.mhx_claims limit 50;', engine)
mx_data.head()

,claim_txn_id,srvc_line_seq,payer_plan_type,procedure_cd,procedure_dt,procedure_mod1_cd,procedure_mod2_cd,procedure_type_cd,patient_id,place_of_srvc,...,diag7_cd,diag8_cd,diag9_cd,billing_prov_id,service_units,claim_proc_dt,revenue_cd,performing_prov_npi,billing_npi,srvc_year
0,5.468000e+14,1.0,Third Party,88175,2011-02-14,None,None,MX,128375004.0,11,...,None,None,None,000000000,1.0,2011-03-05,None,010709650,000000000,2011
1,5.961700e+14,1.0,Third Party,99213,2011-05-06,None,None,MX,219487692.0,11,...,None,None,None,000000000,1.0,2011-05-17,None,030417049,000000000,2011
2,5.964100e+14,1.0,Third Party,88305,2011-01-06,None,None,MX,344005556.0,81,...,None,None,None,000000000,1.0,2011-05-20,None,050528715,000000000,2011
3,8.452502e+13,10.0,Third Party,85610,2011-09-07,None,None,HX,123488009.0,4,...,None,None,None,0,1.0,2011-10-07,0305,1093791071,0,2011
4,6.092600e+14,2.0,Third Party,82550,2011-06-06,None,None,MX,230685242.0,11,...,None,None,None,000000000,1.0,2011-06-18,None,030417049,000000000,2011


### Group data by payer_plan_type with counts of rows

In [8]:
mx_data.groupby(['payer_plan_type']).count()

,claim_txn_id,srvc_line_seq,procedure_cd,procedure_dt,procedure_mod1_cd,procedure_mod2_cd,procedure_type_cd,patient_id,place_of_srvc,srvc_from_dt,...,diag7_cd,diag8_cd,diag9_cd,billing_prov_id,service_units,claim_proc_dt,revenue_cd,performing_prov_npi,billing_npi,srvc_year
payer_plan_type,,,,,,,,,,,,,,,,,,,,,
Medicaid,3,3,3,3,0,0,3,3,3,3,...,0,0,0,3,3,3,0,3,3,3
Medicare,2,2,2,2,1,0,2,2,2,2,...,0,0,0,2,2,2,0,2,2,2
Third Party,45,45,45,45,5,0,45,45,45,45,...,0,0,0,45,45,45,1,45,45,45


### Group data by payer_plan_type with counts of rows on 1 column

In [12]:
mx_data[['payer_plan_type', 'claim_txn_id']].groupby(['payer_plan_type']).count()

,claim_txn_id
payer_plan_type,
Medicaid,3
Medicare,2
Third Party,45
